This is part of team s_toppo's solusion.  

It is very important to choose which agent to learn,
when imitating other team's replay.  
We found "clustering" these agents gives good result.

Let's take WeKick's replay as an example.  
Thank you WeKick!

In [ ]:
import numpy as np
from numpy import linalg as LA

import matplotlib.pyplot as plt
import pandas as pd
import json
import glob
import seaborn as sns

from tqdm import tqdm

import math

from collections import defaultdict
import collections

In [ ]:
TEAMNAME = "WeKick"
replay_dir = "../input/wekick-small/wekick_small/"

In [ ]:
action_set_v1=[
"idle","left","top_left","top","top_right","right","bottom_right","bottom","bottom_left","long_pass","high_pass","short_pass","shot","sprint","release_direction","release_sprint","sliding","dribble","release_dribble"
]

In [ ]:
json_paths=[]
for path in glob.glob(replay_dir+"*"): 
    json_paths.append(path)
        
print("replay num: {}".format(len(json_paths)))

In [ ]:
# return action list
def create_episode_dict(json_path):
    act_lis =[]
    
    json_open = open(json_path, 'r')
    json_load = json.load(json_open)
    
    sub_id = int(json_path.split("/")[-1].split("_")[0])
    
    for frame in range(len(json_load["steps"])-1):
        if TEAMNAME in json_load["info"]["TeamNames"][0]:
            team=0
        elif TEAMNAME in json_load["info"]["TeamNames"][1]:
            team=1
        else:
            raise BaseException("teamname{} not found!".format(TEAMNAME))

        raw = json_load["steps"][frame][team]["observation"]["players_raw"][0]
        action = json_load["steps"][frame+1][team]["action"][0]
        
        act_lis.append(action)

    
    return act_lis,sub_id

In [ ]:
from joblib import Parallel, delayed
sub = Parallel(n_jobs=-1, verbose=10)( [delayed(create_episode_dict)(j) for j in json_paths] )

In [ ]:
subs = defaultdict(list)

for act_lis,sub_id in sub:
    subs[sub_id].extend(act_lis)

In [ ]:
sub_action_prob= defaultdict(list)

for subid, actions in subs.items():
    c = collections.Counter(actions)
    
    for act in range(19):
        sub_action_prob[subid].append(c[act]/len(actions))

In [ ]:
act_df = pd.DataFrame(sub_action_prob)
act_df.index = action_set_v1
act_df = act_df.reindex(sorted(act_df.columns), axis=1)
act_df

In [ ]:
plt.figure(figsize = (15,5))
plt.plot(act_df)
plt.xticks(rotation=45)
plt.grid()

In [ ]:
df_corr = act_df.corr()
plt.figure(figsize = (8,8))
sns.heatmap(df_corr, square=True,vmax=1, vmin=-1, center=0)

In [ ]:
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import dendrogram, linkage

corr_condensed =squareform(1-df_corr)

z = linkage(corr_condensed, method='average')
dn = dendrogram(z,  leaf_rotation=70)

Now we can see 3 "clusters"